# Westeros Tutorial - Introducing Sankey diagrams

Sankey diagrams are a useful technique to visualize energy flow accounts.
This tutorial demonstrates how to produce Sankey diagrams from the solution of a MESSAGEix Scenario object, using features provided by [`plotly`](https://plotly.com/python/) via [`pyam-iamc`](https://pyam-iamc.readthedocs.io).


**Pre-requisites**
- You have the *MESSAGEix* framework installed and working.
  In particular, you should have installed `message_ix[report,sankey]`, which installs the dependencies `pyam` and `plotly`.
- Complete tutorials Part 1 (`westeros_baseline.ipynb`) and “Introducing Reporting” (`westeros_report.ipynb`).

We start as usual by connecting to a database and loading the solved "baseline" scenario of the "Westeros Electified" MESSAGE model.
(Note that we do not `clone()` the scenario here because we do not intend to make any changes to it.)

In [ ]:
import ixmp

from message_ix import Scenario

mp = ixmp.Platform()

try:
    scenario = Scenario(mp, model="Westeros Electrified", scenario="baseline")

    # Ensure the scenario has a solution
    if not scenario.has_solution():
        scenario.solve(quiet=True)
except ValueError:
    # The scenario doesn't exist → use a utility function to create it
    from message_ix.testing import make_westeros

    scenario = make_westeros(mp, solve=True, quiet=True)

Next, we create the `Reporter` object from the solved scenario:

In [2]:
from message_ix.report import Reporter

rep = Reporter.from_scenario(
    scenario,
    # Reporter uses the Python package 'pint' to handle units.
    # "-"", used in the Westeros tutorial, is not a defined SI
    # unit. We tell the Reporter to replace it with ""
    # (unitless) everywhere it appears.
    units={"replace": {"-": ""}},
)

## The `add_sankey()` method

The code uses [`pyam.figures.sankey()`](https://pyam-iamc.readthedocs.io/en/stable/api/plotting.html#pyam.figures.sankey) under the hood which (as of `pyam-iamc` version 3.0.0) supports only one year (MESSAGE time period) and one region (MESSAGE `node`).
Our model is already a single-node model, so we use its one node, and choose to prepare our first Sankey diagram for the **year 700**:


In [ ]:
key = rep.add_sankey(year=700, node="Westeros")
key

This returns a *key*.
As explained in the “Introducing Reporting” tutorial, nothing has happened yet; no data has been retrieved from the Scenario.
The key identifies a task that will trigger all these steps and return the created diagram.
Let's now do that:


In [ ]:
fig = rep.get(key)
type(fig)

The diagram is created!
It is a `plotly.Figure` object.
A Jupyter notebook, like this one, can provide interactive display of this figure:

In [ ]:
fig

This diagram alternates between showing `{technology}|{mode}` (for example: `coal_ppl|standard`) and `{level}|{commodity}` (for example, `secondary|electricity`).
By mousing over the colored areas, we can see that:

- 61.1 units of (level=secondary, commodity=electricity) are produced in (year=700, node=Westeros); of these, 47.4 units are supplied by (technology=coal_ppl, mode=standard) and 13.7 units are supplied by (technology=wind_ppl, mode=standard).
- All of the (secondary, electricity) is consumed as an input to (technology=grid, mode=standard).
- …and so on.

## Simplifying the diagram

Large models like [`MESSAGEix-GLOBIOM`](https://docs.messageix.org/models) can include hundreds of (technology, mode) and (level, commodity) combinations.
You can imagine that this diagram could get very crowded!
To exclude flows we are not interested in, we can use the `exclude` parameter of `add_sankey()`.


In [ ]:
key2 = rep.add_sankey(year=700, node="Westeros", exclude=["wind_ppl|standard"])
key2

Notice this key is different from the previous key.
This allows to prepare multiple diagrams, and later generate one or more of them, without conflict.

Next, we can display the figure as before:

In [ ]:
rep.get(key2)

Compare this diagram to the first one and notice that `wind_ppl|standard` does not appear any more.

You can pick any variable for this, even if it's in the middle of the overall flow!
And, for any scenario like this one with multiple periods, you can pick other years, too:


In [ ]:
key3 = rep.add_sankey(year=720, node="Westeros", exclude=["final|electricity"])
print(key3)
rep.get(key3)

Omitting `final|electricity` splits this Sankey diagram in two, so Plotly automatically arranges the two parts on top of one another.

## Under the hood

This section gives a step-by-step explanation of the atomic tasks that are prepared by `add_sankey()`.
You may wish to read this section to get a better understanding of how the code operates, or if you want to build your own code to do something different.

The function we want to use, `pyam.figures.sankey()`, takes two arguments: `df` and `mapping`.

After calling `Reporter.from_scenario()`, `rep` already has keys for `in::pyam` and `out::pyam`.
These give, respectively the total (level, commodity) inputs to, and outputs from, each (technology, mode), in the IAMC data structure and as a `pyam.IamDataFrame` object.

The first step is to concatenate these two objects together:

In [ ]:
from genno.operator import concat

df_all = concat(rep.get("in::pyam"), rep.get("out::pyam"))
df_all

…and then select the one year to be plotted:

In [ ]:
df = df_all.filter(year=700)
df

Next, to prepare the `mapping` argument, we use the function `message_ix.tools.map_for_sankey()`:

In [ ]:
from message_ix.tools.sankey import map_for_sankey

mapping = map_for_sankey(
    df,
    node="Westeros",
    exclude=["wind_ppl|standard"],
)
mapping

Finally, we generate the plot:

In [ ]:
from pyam.figures import sankey

fig = sankey(df=df, mapping=mapping)
fig

We can see this is the same as the second example diagram in the tutorial.

We can also visualize the steps created by `add_sankey()`:

In [ ]:
print(rep.describe(key2))

This also shows how the core MESSAGE parameters `input` and `output`, and solution variable `ACT`, are retrieved, multiplied, summed on some dimensions, and mapped in the the IAMC data structure understood by `pyam`, leading up to the `concat()` step with which we started this section.

Lastly, as always, please do not forget to close the database 😉

In [14]:
mp.close_db()